In [10]:
from tokenizers import Tokenizer, pre_tokenizers, models
from thai2transformers.tokenizers import CustomPreTokenizer, WordLevelTrainer

#newmm
from pythainlp.tokenize import word_tokenize, syllable_tokenize

#sefr cut
import sefr_cut
sefr_cut.load_model(engine='ws1000')
def pingcut(text):
    return sefr_cut.tokenize(text)[0]

#parameter
tokenize_func = syllable_tokenize #word_tokenize, syllable_tokenize, pingcut (very slow on CPU)
texts = ['โรนัลโดเขาได้เล่นกับทีม','โปรตุเกสมีโรนัลโด']

from transformers import CamembertTokenizer

#tokenizers version used
import tokenizers
tokenizers.__version__

loading model.....
Success


'0.9.4'

In [ ]:
%%time
#get vocab
trainer = WordLevelTrainer(pre_tokenize_func=tokenize_func, 
                           vocab_size=20, 
                           input_dir='test_folder', 
                           additional_special_tokens=['<unk>','<pad>'])
trainer.count_parallel()
trainer.save_vocab('test.json')

In [ ]:
#create pretokenizer and tokenizer
c = CustomPreTokenizer(tokenize_func)
pre = pre_tokenizers.PreTokenizer.custom(c)
tokenizer = Tokenizer(models.WordLevel.from_file('test.json',unk_token = '<unk>'))
tokenizer.pre_tokenizer = pre

In [ ]:
#simple encoding
tokenizer.encode_batch(texts)

In [ ]:
#padding
# tokenizer.enable_padding(pad_id=1, pad_token="<pad>", length=10)

In [ ]:
#truncation
tokenizer.enable_truncation(max_length=5)

In [ ]:
#done
res = [{'input_ids': i.ids, 'attention_mask': i.attention_mask} for i in tokenizer.encode_batch(texts)]
res

In [ ]:
tokenizer.decode(res[0]['input_ids'])